## IPA for $R(p) := p/\mathbb{E}[A]$

In [ ]:
import math

import numpy as np
import pandas as pd

from scipy.optimize import fsolve, minimize, Bounds, LinearConstraint
from scipy.integrate import quad
from scipy import linspace, meshgrid, arange, empty, concatenate, newaxis, shape

import matplotlib.pyplot as plt

from collections import deque

import time

In [ ]:
# Generates service times for n customers

def generate_service_times(x, n):
    # Exponentially distributed, Mean = 1/x
    service_times = np.random.exponential(1/x, n)
    # Gamma distributed, Mean = x[0]x[1], Variance = x[0]x[1]^2
    # service_times = np.random.gamma(x[0], x[1], n)
    return service_times

In [ ]:
# Functions for F_p, F^{-1}_p, and d/dp F^{-1}_p

def F(x, v, price, model_vars):
    lam = model_vars[0]
    theta_1 = model_vars[1]
    theta_2 = model_vars[2]
    zeta = 0
    if x >= v:
        zeta = 1 - np.exp(-lam*np.exp(-theta_1*price)*((1-np.exp(-theta_2*v))/theta_2 + (x-v)))
    else:
        zeta = 1 - np.exp(-lam*np.exp(-theta_1*price)*(np.exp(-theta_2*(v-x))-np.exp(-theta_2*v))/theta_2)
    return zeta
    
def inverse_F(zeta, v, price, model_vars):
    lam = model_vars[0]
    theta_1 = model_vars[1]
    theta_2 = model_vars[2]
    A = 0
    if zeta >= F(v, v, price, model_vars):
        A = v - (1-np.exp(-theta_2*v))/theta_2 - np.log(1-zeta)*np.exp(theta_1*price)/lam
    else:
        A = (1/theta_2)*np.log(1-(theta_2/lam)*np.exp(theta_1*price+theta_2*v)*np.log(1-zeta))
    return A
    
    
def gradient_inverse_F(zeta, v, gradient_v, price, model_vars):
    lam = model_vars[0]
    theta_1 = model_vars[1]
    theta_2 = model_vars[2]
    gradient_A = 0
    if zeta >= F(v, v, price, model_vars):
        gradient_A = (1 - np.exp(-theta_2*v))*gradient_v - (theta_1/lam)*np.exp(theta_1*price)*np.log(1-zeta)
    else:        
        gradient_A =  (theta_1+theta_2*gradient_v)/(theta_2-lam*np.exp(-theta_1*price-theta_2*v)/np.log(1-zeta))
    return gradient_A

In [ ]:
# Simulates queue given seeds zeta, service times, price, and model variables

# Model will be as follows - First arrival is at time 0. If there are "n" interarrivals, then n+1 customers in total
# This means that simulation ends with one last customer joining, but no more interarrival times simulated

def queue_simulation(zeta, service_times, price, model_vars):
    n_servers = model_vars[3]
    workloads = np.zeros(n_servers)
    
    n_customers = np.size(service_times)
    
    workloads_plus = []
    workloads_minus = []
    interarrival_times = []
###################################################################################################################################################################################   
    for i in range(n_customers-1):
        workloads_minus.append(workloads)
        min_workload_server = workloads.argmin()
        workloads[min_workload_server] += service_times[i]
        workloads_plus.append(workloads)
        
        next_interarrival_time = inverse_F(zeta[i], workloads.min(), price, model_vars)
        interarrival_times.append(next_interarrival_time)
        
        workloads -= next_interarrival_time
        workloads = np.where(workloads < 0, 0, workloads)
        
    workloads_minus.append(workloads)
    workloads[workloads.argmin()] += service_times[n_customers-1]
    workloads_plus.append(workloads)
####################################################################################################################################################################################
    workloads_minus = np.array(workloads_minus)
    workloads_plus = np.array(workloads_plus)
    interarrival_times = np.array(interarrival_times)
    
    return [workloads_minus, workloads_plus, interarrival_times]

In [ ]:
def revenue_gradient_estimator(zeta, interarrival_times, workloads_minus, workloads_plus, price, model_vars):
    lam = model_vars[0]
    theta_1 = model_vars[1]
    theta_2 = model_vars[2]
    n_servers = model_vars[3]
    
    n_interarrivals = np.size(interarrival_times)
    
    estimator_term_1 = np.mean(interarrival_times)
    estimator_term_2 = 0
    
    gradient_workloads = np.zeros(n_servers) # dW vector at time 0, when first arrival occurs
    grad_v = 0
    gradient_virtual_waiting_times = [0] # Corresponding dV/dp at time 0
    
    for i in range(n_interarrivals-1):
        # Update dW vector for the next arrival
        # print("Revenue gradient estimator loop 1: ", (zeta[i], workloads_plus[i].min(), grad_v))
        change = gradient_inverse_F(zeta[i], workloads_plus[i].min(), grad_v, price, model_vars)
        gradient_workloads = np.where(workloads_plus[i] == 0, 0, gradient_workloads - change)
        
        # Now that we know dW and underline{W} for the next arrival, we can re-evaluate gradient_v
        grad_v = gradient_workloads[workloads_minus[i+1].argmin()]
        gradient_virtual_waiting_times.append(grad_v)
  
    # This loop calculates estimator_term_2
    for i in range(n_interarrivals):
        # print("Revenue gradient estimator loop 2: ", (zeta[i], workloads_plus[i].min(), gradient_virtual_waiting_times[i]))
        estimator_term_2 += (1/n_interarrivals)*gradient_inverse_F(zeta[i], workloads_plus[i].min(), gradient_virtual_waiting_times[i], price, model_vars)
        
    revenue_grad_est = 1/estimator_term_1 - price*estimator_term_2/estimator_term_1**2
    
    return revenue_grad_est

In [ ]:
def implement_IPA(n_iters, initial_n_customers, initial_price, model_vars):
    
    price = initial_price
    price_iterates = [initial_price]
    
    for i in range(1, n_iters+1):
        
        # Generating data
        n_customers = (int)(initial_n_customers * math.sqrt(i))
        service_times = generate_service_times(1, n_customers)
        zeta = np.random.random(n_customers-1)
        
        # Queue simulation
        [workloads_minus, workloads_plus, interarrival_times] = queue_simulation(zeta, service_times, price, model_vars)
        
        # Gradient estimator
        revenue_grad_est = revenue_gradient_estimator(zeta, interarrival_times, workloads_minus, workloads_plus, price, model_vars)
        
        # Gradient ascent
        learning_rate = 10/np.power(i, 1)
        price = price + learning_rate*revenue_grad_est
    
        price_iterates.append(price)
        
        # print(i)
    
    return price_iterates

In [ ]:
# Simulation

n_iters = 10
N_0 = 20
initial_price = 10
model_vars = [20, 0.1, 0.2, 5]

price_iterates = implement_IPA(n_iters, N_0, initial_price, model_vars)

In [ ]:
## Plots

optimal_price = 26.6

plt.plot(price_iterates, color = "blue")
plt.axhline(optimal_price, color = "red", linestyle="dashed")

plt.xlabel("Iterate")
plt.ylabel("Price")
plt.title("IPA based gradient descent")
plt.savefig("IPA.pdf")

## Obtaining $p^*$ via simulation

In [ ]:
def simulated_average_revenue(n_customers, price, arrival_times, service_times, model_vars):
    lam = model_vars[0]
    theta_1 = model_vars[1]
    theta_2 = model_vars[2]
    n_servers = model_vars[3]
    
    server_workloads = np.zeros(n_servers)
    total_collections = 0
##################################################################################################################################   
    for i in range(n_customers):
        joining_prob = np.exp(-theta_1*price-theta_2*server_workloads.min())
        joining_decision = np.random.binomial(1, joining_prob)
        
        if joining_decision == 1:
            total_collections += price
            
        if i != n_customers-1:
            min_workload_server = server_workloads.argmin()
            server_workloads[min_workload_server] += service_times[i]
            
            server_workload = max(0, server_workload - (arrival_times[i+1]-arrival_times[i]))
    
    average_revenue_per_unit_time = total_collections/arrival_times[-1]
    
    return average_revenue_per_unit_time

In [ ]:
# Check optimal price via simulation

model_vars = [10, 0.1, 0.2, 5]

prices = np.linspace(14.1, 16, 20)
simulated_revenues = []

for p in prices:
    n_iter = 100
    n_customers = 100000
    
    average_average_revenue = 0
    for i in range(n_iter):
        interarrival_times = np.random.exponential(1/model_vars[0], n_customers)
        arrival_times = np.cumsum(interarrival_times)
        service_times = np.random.exponential(1, n_customers)
    
        average_average_revenue += (1/n_iter)*simulated_average_revenue(n_customers, p, arrival_times, service_times, model_vars)
    
    print(p)
    
    simulated_revenues.append(average_average_revenue)

In [ ]:
optimal_price = 0
optimal_revenue = 0

for i in range(len(prices)):
    if simulated_revenues[i] > optimal_revenue:
        optimal_price = prices[i]
        optimal_revenue = simulated_revenues[i]

print(optimal_price)

In [ ]:
plt.plot(prices, simulated_revenues, color="red")
plt.axyline(optimal_price, color="blue", "dashed")
plt.xlabel("p")
plt.ylabel("R(p)")
plt.title("Average revenue per unit time vs price")
plt.savefig("Revenue_versus_price.pdf")